## DASK groupby and sorting on Hyperplane 

In [1]:
import warnings
import os
import sys
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd
from dask.distributed import Client
from typing import List, Set, Dict, Tuple, Optional
import types
from google.cloud import storage
from tqdm.notebook import tqdm

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt 
%matplotlib inline

pd.options.display.max_rows = 999
warnings.filterwarnings('ignore')

from hyperplane import notebook_common as nc

###  set parameters 
A parameter cell is marked by clicking on the cog symbol at the top right corner.
Variables set in this cell will then become environment variable and can be overwritten by injected parameters when running notebook as a pipeline job.


In [2]:
year = "199*"

In [3]:
data_url = f"s3://dask-data/airline-data/{year}.csv"
data_url

's3://dask-data/airline-data/199*.csv'

#### Below is the one liner to scale up the job to kubernetes

In [4]:
client, cluster = nc.initialize_cluster(
        nprocs=3,
        nthreads=5,
        ram_gb_per_proc = 4,
        cores_per_worker=15,
        scheduler_deploy_mode="remote",
        num_workers = 3
    )


👉 Hyperplane: selecting worker node pool
👉 Hyperplane: selecting scheduler node pool
Creating scheduler pod on cluster. This may take some time.
👉 Hyperplane: spinning up a dask cluster with a scheduler as a standalone container.
👉 Hyperplane: In a few minutes you'll be able to access the dashboard at https://ds.hyperplane.dev/dask-cluster-a71138c2-92c7-4870-b200-69e10989c489/status
👉 Hyperplane: to get logs from all workers, do `cluster.get_logs()`


In [5]:
%%time
df = dd.read_csv(data_url, 
                 storage_options = {'anon': True},
                usecols = ['DepTime','FlightNum','DepDelay','Origin', 'Dest','Distance'],
                dtype={'Distance': 'float64',
                      'DepTime':'float64',
                      'FlightNum':'int64',
                      'DepDelay':'float64',
                      'Dest':'object',
                      }, 
                encoding = "ISO-8859-1")
df_sort = df.groupby('Origin').apply(lambda x : x.nlargest(n = 10, columns = 'Distance'))
df_sort.compute()

CPU times: user 798 ms, sys: 70.9 ms, total: 869 ms
Wall time: 18.8 s


DepTime  FlightNum  DepDelay Origin Dest  Distance
Origin                                                          
ROA    65653   1140.0       1777       0.0    ROA  LGA     405.0
       65654   1142.0       1777       2.0    ROA  LGA     405.0
       65655   1140.0       1777       0.0    ROA  LGA     405.0
       65656      NaN       1777       NaN    ROA  LGA     405.0
       65657   1140.0       1777       0.0    ROA  LGA     405.0
...               ...        ...       ...    ...  ...       ...
ITH    11497    800.0        253       0.0    ITH  PIT     239.0
       11498    800.0        253       0.0    ITH  PIT     239.0
       11499    800.0        253       0.0    ITH  PIT     239.0
       11500    800.0        253       0.0    ITH  PIT     239.0
       11501    800.0        253       0.0    ITH  PIT     239.0

[2541 rows x 6 columns]

## Compare with pandas 
- will run **out of memory** for this operation with data from 1999 to 1999 2.5G
- took 21 second for one year of data, if process all 10 years one by one will take at least **3.5mins**

In [1]:
%%time
import pandas as pd
df_pd = pd.read_csv("s3://dask-data/airline-data/1990.csv", 
                    usecols = ['DepTime','FlightNum','DepDelay','Origin', 'Dest','Distance'])
df_sort_pd = df_pd.groupby('Origin').apply(lambda x : x.nlargest(n = 10, columns = 'Distance'))

CPU times: user 9.29 s, sys: 1.56 s, total: 10.8 s
Wall time: 21 s


## close cluster after done
It's a good idea to close the cluster after use. If you forgot to add this cell, don't worry :) Hyperplane will automatically garbage collect the node after it detects it's being idle for a bit


In [8]:
cluster.close()